In [7]:
from pymongo import MongoClient
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from textblob import TextBlob
import spacy
from spacy import displacy

import speech_recognition as sr


In [8]:
import en_core_web_sm
import pyaudio

ModuleNotFoundError: No module named 'en_core_web_sm'

In [53]:
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/pptrv/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/pptrv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/pptrv/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Bringing Data

In [54]:
client = MongoClient("mongodb://localhost/SW")
db = client.get_database()
collection = db["star_wars"]

## Assigning Variables & DF

In [55]:
characters = collection.distinct("Character")
characters

['ASTRO-OFFICER',
 'AUNT',
 'BARTENDER',
 'BASE',
 'BEN',
 'BERU',
 'BIGGS',
 'CAMIE',
 'CAPTAIN',
 'CHIEF',
 'COMMANDER',
 'CONTROL',
 'CREATURE',
 'DEAK',
 'DEATH',
 'DODONNA',
 'FIRST',
 'FIXER',
 'GANTRY',
 'GOLD',
 'GREEDO',
 'HAN',
 'HUMAN',
 'IMPERIAL',
 'INTERCOM',
 'JABBA',
 'LEIA',
 'LUKE',
 'MAN',
 'MASSASSI',
 'MOTTI',
 'OFFICER',
 'OWEN',
 'PORKINS',
 'REBEL',
 'RED',
 'SECOND',
 'TAGGE',
 'TARKIN',
 'TECHNICIAN',
 'THREEPIO',
 'TROOPER',
 'VADER',
 'VOICE',
 'WEDGE',
 'WILLARD',
 'WINGMAN',
 'WOMAN']

In [56]:
charac_and_dialog_df = pd.DataFrame(list(collection.find()))
charac_and_dialog_df.head()

,_id,Line_order,Character,Dialogue
0,5fc517d1acba21496186d93b,4,THREEPIO,What's that?
1,5fc517d1acba21496186d93c,3,THREEPIO,There'll be no escape for the Princess this time.
2,5fc517d1acba21496186d93d,2,THREEPIO,We're doomed!
3,5fc517d1acba21496186d93e,12,VADER,Where are those transmissions you intercepted?
4,5fc517d1acba21496186d93f,13,REBEL,"OFFICER"" ""We intercepted no transmissions. Aaa..."


In [57]:
charac_and_dialog_df = charac_and_dialog_df[["Character", "Dialogue"]]
charac_and_dialog_df.head()

,Character,Dialogue
0,THREEPIO,What's that?
1,THREEPIO,There'll be no escape for the Princess this time.
2,THREEPIO,We're doomed!
3,VADER,Where are those transmissions you intercepted?
4,REBEL,"OFFICER"" ""We intercepted no transmissions. Aaa..."


## Sentiment Analysis

#### NLTK

In [58]:
sia = SentimentIntensityAnalyzer()

In [59]:
def sentimentAnalysisPolarity(sentence):
    sia = SentimentIntensityAnalyzer()
    polarity = sia.polarity_scores(sentence)
    return polarity  

In [1]:
def sentimentAnalysisCompound(sentence):
    sia = SentimentIntensityAnalyzer()
    polarity = sia.polarity_scores(sentence)
    pol = polarity['compound']
    return pol

In [61]:
charac_and_dialog_df["sentiment_compound"] = charac_and_dialog_df["Dialogue"].apply(sentimentAnalysisCompound)
charac_and_dialog_df.head()

,Character,Dialogue,sentiment_compound
0,THREEPIO,What's that?,0.0000
1,THREEPIO,There'll be no escape for the Princess this time.,-0.1280
2,THREEPIO,We're doomed!,-0.6696
3,VADER,Where are those transmissions you intercepted?,0.0000
4,REBEL,"OFFICER"" ""We intercepted no transmissions. Aaa...",-0.2960


In [62]:
charac_and_dialog_df["sentiment_polarity"] = charac_and_dialog_df["Dialogue"].apply(sentimentAnalysisPolarity)
charac_and_dialog_df.head()

,Character,Dialogue,sentiment_compound,sentiment_polarity
0,THREEPIO,What's that?,0.0000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,THREEPIO,There'll be no escape for the Princess this time.,-0.1280,"{'neg': 0.202, 'neu': 0.642, 'pos': 0.156, 'co..."
2,THREEPIO,We're doomed!,-0.6696,"{'neg': 0.818, 'neu': 0.182, 'pos': 0.0, 'comp..."
3,VADER,Where are those transmissions you intercepted?,0.0000,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,REBEL,"OFFICER"" ""We intercepted no transmissions. Aaa...",-0.2960,"{'neg': 0.145, 'neu': 0.855, 'pos': 0.0, 'comp..."


In [63]:
grouped = charac_and_dialog_df.groupby('Character').mean().reset_index()
grouped_sorted = grouped.sort_values("sentiment_compound", ascending = False)
grouped_sorted

,Character,sentiment_compound
5,BERU,0.318200
13,DEAK,0.240100
9,CHIEF,0.233700
46,WINGMAN,0.200950
7,CAMIE,0.170600
1,AUNT,0.086817
6,BIGGS,0.077529
21,HAN,0.076799
26,LEIA,0.063054
10,COMMANDER,0.042900


#### TextBlob